# Get Nightlight Data from Google Earth Engine using GEEMap

In [1]:
import ee
import geemap

## Authenticate & Initialize GEE

Requires a [Google Cloud Project](https://console.cloud.google.com/projectcreate) and to enable the [Earth Engine API](https://console.cloud.google.com/apis/api/earthengine.googleapis.com) for the project. Find detailed instructions [here](https://book.geemap.org/chapters/01_introduction.html#earth-engine-authentication).

In [2]:
ee.Initialize()

## Create a GEEMap Object

In [3]:
m = geemap.Map(
    center=[-5, 15], 
    zoom=3, 
    basemap = 'Esri.WorldImagery'
)

## Add Layers to the Map

In [4]:
# add nightlights median
# https://developers.google.com/earth-engine/datasets/catalog/NOAA_VIIRS_DNB_MONTHLY_V1_VCMSLCFG
dataset_night = ee.ImageCollection('NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG') \
                  .filter(ee.Filter.date('2022-01-01', '2023-11-01'))
nighttime = dataset_night.select('avg_rad')
image_night = nighttime.median()
nighttimeVis = {'min': 0.0, 'max': 2.0}
# m.addLayer(image_night, nighttimeVis, 'Nighttime', False)


In [5]:
# add WorldPop population density layer
# https://developers.google.com/earth-engine/datasets/catalog/WorldPop_GP_100m_pop#description
dataset_pop = ee.ImageCollection('WorldPop/GP/100m/pop') \
                  .filter(ee.Filter.date('2020-01-01', '2020-12-31'))
pop = dataset_pop.select('population')
image_pop = pop.median()
popVis = {
    'min': 0.0, 
    'max': 20.0,
    'palette': ['24126c', '1fff4f', 'd4ff50'],
    'opacity': 0.5
}
# m.addLayer(image_pop, popVis, 'Population', False)


In [6]:
# overlay country boundaries with white borders
countries = ee.FeatureCollection('FAO/GAUL/2015/level0')
# styleParams = {
#   'fillColor': '00000000',
#   'color': 'ffffff',
#   'width': 1.0,
#   'lineType': 'solid'
# }
# style = {'color': 'ffffff82', 'width': 1, 'lineType': 'solid', 'fillColor': 'ffffff00'}
style = {'color': 'ffffffff', 'width': 2, 'lineType': 'solid', 'opacity': 1}
# m.addLayer(countries, style, 'Countries', False)

sl = countries.filter(ee.Filter.eq('ADM0_NAME', 'Sierra Leone'))

# note: this isn't styling the countries correctly
# manually change the country layer to have a transparent fill on the interactive map
# note do not rerun these cells multiple times, seems to freeze the map

In [7]:
# add place names
# https://developers.google.com/earth-engine/datasets/catalog/FAO_GAUL_2015_level0
m.add_basemap('CartoDB.DarkMatterOnlyLabels')
# m.remove_layer('CartoDB.VoyagerOnlyLabels') # doesn't work

In [8]:
# display the map zoomed into Sierra Leone
m.setCenter(-11.779889, 8.460555, 8)

## Get the Landcover Data from Google Earth Engine

In [9]:
# pull in a global high resolution land cover dataset
# https://developers.google.com/earth-engine/datasets/catalog/ESA_WorldCover_v200
landcover = ee.ImageCollection('ESA/WorldCover/v200').first()

landcover_sl = landcover.clip(sl)

visualization = {
  'bands': ['Map'],
}

# m.addLayer(landcover_sl, visualization, 'Landcover Sierra Leone', False)

# inspect this image
print(landcover_sl.getInfo())
# inspect the bands of landcover_sl
print(landcover_sl.bandNames().getInfo())
# inspect the values of the band 'Map'
print(landcover_sl.select('Map').getInfo())



{'type': 'Image', 'bands': [{'id': 'Map', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'dimensions': [4320000, 1728000], 'crs': 'EPSG:4326', 'crs_transform': [8.333333333333333e-05, 0, -180, 0, -8.333333333333333e-05, 84]}], 'version': 1699537784392512, 'id': 'ESA/WorldCover/v200/2021', 'properties': {'Map_class_names': ['Tree cover', 'Shrubland', 'Grassland', 'Cropland', 'Built-up', 'Bare / sparse vegetation', 'Snow and ice', 'Permanent water bodies', 'Herbaceous wetland', 'Mangroves', 'Moss and lichen'], 'system:time_start': 1609455600000, 'system:time_end': 1640991600000, 'Map_class_palette': ['006400', 'ffbb22', 'ffff4c', 'f096ff', 'fa0000', 'b4b4b4', 'f0f0f0', '0064c8', '0096a0', '00cf75', 'fae6a0'], 'Map_class_values': [10, 20, 30, 40, 50, 60, 70, 80, 90, 95, 100], 'system:asset_size': 109661138988, 'system:index': '2021'}}
['Map']
{'type': 'Image', 'bands': [{'id': 'Map', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255

## Select Just for Dark Areas that are Trees and Water far away from Built Areas

In [10]:

# create a mask for the landcover
landcover_sl_built_mask = landcover_sl.eq(50)
landcover_sl_built = landcover_sl.updateMask(landcover_sl_built_mask)
# m.addLayer(landcover_sl_built, {'palette': 'red'}, 'Built')

# create a 10km buffer around built areas
built_buffer = landcover_sl_built.focal_max(10000, 'circle', 'meters')
m.addLayer(built_buffer, {'palette': 'red'}, 'Built buffer', False)

# create a mask for the built_buffer
# need to unmask it to convert areas outside of mask from nodata to 0
rural_mask = built_buffer.eq(50).unmask(0).eq(0)
# select everywhere in Sierra Leone outside of the built buffer
landcover_sl_rural = landcover_sl.updateMask(rural_mask)
m.addLayer(landcover_sl_rural, {'palette': 'brown'},  'Rural landcover', False)



In [11]:
# select just treed areas and water areas from landcover_sl_rural
rural_trees_mask = landcover_sl_rural.eq(10)
rural_water_mask = landcover_sl_rural.eq(80)

rural_trees = landcover_sl_rural.updateMask(rural_trees_mask)
rural_water = landcover_sl_rural.updateMask(rural_water_mask)

# add the rural tree and water layers
m.addLayer(rural_trees, {'palette': 'green'}, 'Rural tree cover', False)
m.addLayer(rural_water, {'palette': 'blue'}, 'Rural water', False)

## Sample These Areas

In [17]:
# sample 50 random points from the rural tree cover
rural_trees_points = rural_trees.sample(
    region=sl,
    scale=3000,
    numPixels=50,
    seed=44,
    dropNulls=True,
    geometries=True
)

# convert into a feature collection of points
rural_trees_fc = ee.FeatureCollection(rural_trees_points)

m.addLayer(
    rural_trees_fc, 
    {'color': '00ff00', 'pointSize': 10}, 
    'Rural tree points', 
    True
)



In [ ]:
# repeat for rural water
rural_water_points = rural_water.sample(
    region=sl,
    scale=3000,
    numPixels=50,
    seed=44,
    dropNulls=True,
    geometries=True
)

rural_water_fc = ee.FeatureCollection(rural_water_points)

m.addLayer(
    rural_water_fc, 
    {'color': '0000ff', 'pointSize': 10}, 
    'Rural water points', 
    True
)

## Create a 1km Buffer Around the Sampled Points

## Get the Nightlights Values for each buffer

## Export the Values to a CSV

## Visualize the Results on a Map

In [13]:
# map the map taller
m.layout.height = '1200px'
m

Map(center=[8.460555, -11.779889], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Sear…